In [12]:

import time # std module
import pyvisa as visa # http://github.com/hgrecco/pyvisa
import matplotlib.pyplot as plt # http://matplotlib.org/
import numpy as np # http://www.numpy.org/



In [13]:
visa_address = 'TCPIP::192.168.50.25::INSTR'

rm = visa.ResourceManager()
scope = rm.open_resource(visa_address)
scope.timeout = 10000 # ms
scope.encoding = 'latin_1'
scope.read_termination = '\n'
scope.write_termination = None
scope.write('*cls') # clear ESR

print(scope.query('*idn?'))

TEKTRONIX,MSO46B,C065659,CF:91.1CT FV:2.6.38.1348


In [6]:
scope.write('*rst') # reset
t1 = time.perf_counter()
r = scope.query('*opc?') # sync
t2 = time.perf_counter()
print('reset time: {}'.format(t2 - t1))

reset time: 0.2033433999999943


In [8]:
scope.write('autoset EXECUTE') # autoset
t3 = time.perf_counter()
r = scope.query('*opc?') # sync
t4 = time.perf_counter()
print('autoset time: {} s'.format(t4 - t3))

autoset time: 1.6603302999999983 s


In [9]:
# io config
scope.write('header 0')
scope.write('data:encdg SRIBINARY')
scope.write('data:source CH1') # channel
scope.write('data:start 1') # first sample
record = int(scope.query('horizontal:recordlength?'))
scope.write('data:stop {}'.format(record)) # last sample
scope.write('wfmoutpre:byt_n 1') # 1 byte per sample

17

In [15]:

print(scope.query('*IDN?'))
scope.write("HEADER 0")
scope.write("DATA:SOUR CH1")
scope.write("DAT:ENC SRI")   # Signed Binary Format, LSB order
scope.write("DAT:WIDTH 1")

scope.write("DAT:START 1")
scope.write("DAT:STOP 1e10") # Set data stop to max
#recordLength = int(scope.query("WFMO:NR_P?"))  # Query how many points are actually available
#scope.write("DAT:STOP {0}".format(recordLength)) # Set data stop to match points available

# Fetch horizontal scaling factors
xinc = float(scope.query("WFMO:XINCR?"))
xzero = float(scope.query("WFMO:XZERO?"))
pt_off = int(scope.query("WFMO:PT_OFF?"))

# Fetch vertical scaling factors
ymult = float(scope.query("WFMO:YMULT?"))
yzero = float(scope.query("WFMO:YZERO?"))
yoff = float(scope.query("WFMO:YOFF?"))

# Fetch waveform data
scope.write("curve?")

# Data is sent back with ieee defined header.  ie. #41000<binary data bytes>\n
# PyVISA read_binary_values() method will automatically read and parse the ieee block header so you don't have to.
rawData = scope.read_binary_values(datatype='b', is_big_endian=False, container=np.ndarray, header_fmt='ieee', expect_termination=True)
dataLen = len(rawData)

# Create numpy arrays of floating point values for the X and Y axis
t0 = (-pt_off * xinc) + xzero
xvalues = np.ndarray(dataLen, np.float)
yvalues = np.ndarray(dataLen, np.float)
for i in range(0,dataLen):
    xvalues[i] = t0 + xinc * i # Create timestamp for the data point
    yvalues[i] = float(rawData[i] - yoff) * ymult + yzero # Convert raw ADC value into a floating point value

# Save data to disk in binary format.  Can be read back from file using np.load()
xvalues.dump(r"./xvalues_numpy_array.dat")
yvalues.dump(r"./yvalues_numpy_array.dat")

scope.close()
rm.close()

TEKTRONIX,MSO46B,C065659,CF:91.1CT FV:2.6.38.1348


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.